# Dynamic Topic Model (DTM)

El siguiente notebook toma como input **robos_prose_celan_2.0.xlsx** y realiza lo siguiente:
1. Postprocesa los documentos, eliminando palabras con frecuencia menor a 10 y selecciona relatos entre 2011-2016.
2. Transforma los documentos a bag of words deacuerdo al formato de gensim y segmenta el corpus por mes.
3. Entrena DTM para $K=2,\ldots, 10,$ y guarda los modelos.

## 1. Librerías

In [3]:
import numpy as np
import pandas as pd 
from time import time 

#Topic model
import gensim
from gensim.models.ldaseqmodel import LdaSeqModel
from gensim.models.wrappers import DtmModel
from gensim.corpora import Dictionary
from gensim.corpora.mmcorpus import MmCorpus

import re
import ast
from collections import Counter

#Para no mostrar warnings
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

## 2. Post Procesamiento

Importar datos procesados, eliminación de palabras poco frecuentes, particionar corpus por año.

In [4]:
#Importar relatos procesados
df0 = pd.read_excel('robos_prose_clean_2.0.xlsx')
df0.head()

,sin_fecha_siniestro,sin_relato,sin_relato_clean
0,29-08-2013 0:0,DEJE ESTACIONADO LA CAMIONETA POR MAS MENOS PO...,"['deje', 'estacionado', 'camioneta', 'menos', ..."
1,05-09-2013 0:0,DEJE LA CAMIONETA ESTACIONADA EL DIA 04/09 Y H...,"['deje', 'camioneta', 'estacionada', 'sali', '..."
2,05-09-2013 0:0,DEJE EL VEHICULO ESTACIONADO APROXIMADAMENTE A...,"['deje', 'estacionado', 'aproximadamente', 'lu..."
3,05-09-2013 0:0,DEJO VEHICULO PARA HACER CONSULTAS EN LUVAL Y ...,"['dejo', 'hacer', 'consultas', 'duval', 'volve..."
4,06-09-2013 0:0,EL VIERNES 06 DE SEPTIEMBRE SALE DEL RESTORAN...,"['viernes', 'septiembre', 'sale', 'restoran', ..."


In [5]:
#Obtener año y mes de una fecha
def date_year(fecha):
    y=int(re.split('\W+', fecha)[2])
    return y

def date_month(fecha):
    m = int(re.split('\W+', fecha)[1])
    return m

In [9]:
#Frecuencia de cada palabra que sobrevivio al pre-procesamiento

t1=time()

corpus = [ast.literal_eval(doc) for doc in df0['sin_relato_clean']]
words = [word for doc in corpus for word in doc]
freq_per_words = Counter(words)

dfreq = pd.DataFrame()
dfreq['Palabra'] = freq_per_words .keys()
dfreq['Frecuencia'] = freq_per_words .values()

t2=time()
print(t2-t1)

1.9149887561798096


In [10]:
print('Tamaño del vocabulario: ', len(dfreq))

Tamaño del vocabulario:  30284


In [11]:
dfreq.head()

,Palabra,Frecuencia
0,deje,12089
1,estacionado,27620
2,camioneta,6623
3,menos,325
4,media,441


In [12]:
dfreq.tail()

,Palabra,Frecuencia
30279,furgonque,1
30280,rosael,1
30281,aprobada,1
30282,leonora,1
30283,sechura,1


In [13]:
#Postprocesamiento
t1 = time()
voc_freq = [*dfreq[dfreq['Frecuencia']>=10]['Frecuencia']]
vocabulary = [*dfreq[dfreq['Frecuencia']>=10]['Palabra']]

newcorpus = corpus
for i, doc in enumerate(corpus):
    for j, word in enumerate(doc):
        if freq_per_words[word]<10:  
            newcorpus[i][j]=''
    newcorpus[i] = [elem for elem in newcorpus[i] if elem.strip()]   
    
t2= time()
print(t2-t1)

0.3345823287963867


In [14]:
#Frecuencia de cada palabra que sobrevivio al post-procesamiento
t1=time()

words = [word for doc in newcorpus for word in doc]
freq_per_words = Counter(words)

dfreq = pd.DataFrame()
dfreq['Palabra'] = freq_per_words .keys()
dfreq['Frecuencia'] = freq_per_words .values()

t2=time()
print(t2-t1)

0.13460326194763184


In [15]:
dfreq.sort_values('Frecuencia', ascending=False).tail()

,Palabra,Frecuencia
5085,piquetes,10
1649,unos,10
4483,acción,10
3818,acorralan,10
2221,mochilas,10


In [16]:
print('Tamaño del vocabulario: ', len(dfreq))

Tamaño del vocabulario:  5431


In [17]:
df0['year'] = df0.apply(lambda x: date_year(x['sin_fecha_siniestro']), axis=1)
df0['month'] = df0.apply(lambda x: date_month(x['sin_fecha_siniestro']), axis=1)

#Nuevo dataframe con los relatos procesados y por año
df = pd.DataFrame({'corpus':newcorpus, 'slice':df0['month'].tolist(), 'year':df0['year'].tolist()})
df = df[(df['year']>=2011) & (df['year']<=2016)]
df.sort_values(['year', 'slice'], ascending=True, inplace=True)
df.reset_index(inplace=True)

## 3. Bag of words

In [ ]:
# Cantidad de relatos por mes
time_slices = df.groupby(['year','slice']).count()['index'].tolist()

#Creamos el diccionario a partir de los textos procesados en el formato que necesita el modelo
dictionary = Dictionary([*df['corpus']])
dictionary.save('dictionary.dict')

#creamos el corpus para darle al modelo (segun el formato de esta libreria)
#El corpus contiene una representacion numerica de los textos, un texto es representada por una lista de tuplas
#donde el primer elemento de la tupla es la id de la palabra y el segundo es su frecuencia de aparición en el texto.
corpus = [dictionary.doc2bow(text) for text in [*df['corpus']]]
MmCorpus.serialize('corpora.mm', corpus)

In [5]:
# Cantidad de relatos  por mes
time_slices = df.groupby(['year','slice']).count()['index'].tolist()

#Relatos procesados
newclean = []

for i in range(len(df)):dd
    newclean.append(df['clean'][i])

dictionary = Dictionary(newclean)

#guardamos nuestro diccionario a partir de los textos procesados
dictionary.save('dictionary.dict')

#creamos el corpus para darle al modelo (segun el formato de esta libreria)
#El corpus contiene una representacion numerica de los textos, un texto es representada por una lista de tuplas
#donde el primer elemento de la tupla es la id de la palabra y el segundo es su frecuencia de aparición en el texto.
corpus = [dictionary.doc2bow(text) for text in newclean]

MmCorpus.serialize('corpora.mm',corpus)

## 4. DTM MODEL

Entrenar DTM para K=2,...,10, luego guardar los modelos.

In [29]:
K= [2, 3, 4, 5, 6, 7, 8, 9, 10]
for k in K:
    ti = time()
    model = DtmModel('dtm-win64.exe', corpus = corpus, time_slices=time_slices, num_topics=k, id2word=dictionary)
    model.save('dtm_{k}.model'.format(k=k))
    tf = time()
    print(tf-ti)

1519.8437383174896
2132.442759037018
2452.542304992676
2648.8207592964172
2943.4718515872955
3064.9098930358887
3232.069714784622
3366.5122475624084
3594.083586215973
